In [74]:
import datetime

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [75]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [76]:
# Removing the first 3 columns as they're not imp

data = data.drop(['RowNumber','CustomerId', 'Surname'],axis=1)


In [77]:
data.head()


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [78]:
# LAbel encoding for categorical variables (male = 1, female = 0)
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [79]:
data['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [81]:
# One Hot Encoding
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [82]:
#geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns = onehot_encoder_geo.get_feature_names_out(['Geography']))

geo_encoded_df=pd.DataFrame(geo_encoder,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [83]:
data = pd.concat([data.drop('Geography',axis = 1), geo_encoded_df],
axis = 1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [84]:
# Saving the encoders
with open ('label_encoder_gender.pkl', 'wb') as file :
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file :
    pickle.dump(onehot_encoder_geo,file)


In [85]:
X = data.drop('Exited',axis = 1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [86]:
with open('scaler.pkl','wb') as file :
    pickle.dump(scaler,file)

# So now everything is ready, on this data our ANN will be trained

In [87]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard


In [88]:
# Building the ANN Model
model = Sequential([
    Dense(64,activation = 'relu', input_shape = (X_train.shape[1],)), # HL1 Connected to i/p layer
    Dense(32,activation = 'relu'), # HL2
    Dense(1,activation = 'sigmoid') # o/p layer
    ]

)


In [89]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                832       
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [90]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate = 0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()
loss

In [91]:
## compile the model
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [92]:
# Setup the tensorboard
import tensorflow.keras.callbacks
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

log_dir = 'logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [93]:
# Early stopping
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience=5, restore_best_weights=True)

In [94]:
history = model.fit(
    X_train,y_train, validation_data=(X_test,y_test), epochs = 100, callbacks = [tensorflow_callback, early_stopping_callback]

)

Epoch 1/100
250/250 [==============================] - 3s 7ms/step - loss: 0.4002 - accuracy: 0.8331 - val_loss: 0.3539 - val_accuracy: 0.8585
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3570 - accuracy: 0.8565 - val_loss: 0.3416 - val_accuracy: 0.8600
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3484 - accuracy: 0.8591 - val_loss: 0.3529 - val_accuracy: 0.8595
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3433 - accuracy: 0.8591 - val_loss: 0.3678 - val_accuracy: 0.8555
Epoch 5/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3402 - accuracy: 0.8639 - val_loss: 0.3469 - val_accuracy: 0.8630
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3382 - accuracy: 0.8590 - val_loss: 0.3459 - val_accuracy: 0.8635
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3333 - accuracy: 0.8648 - val_loss: 0.3551 - val_accuracy: 0.8610

In [95]:
model.save('model.h5')

C:\Users\sanku\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [96]:
# load Tensorboard Extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [97]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 2800), started 22:59:57 ago. (Use '!kill 2800' to kill it.)